# In this post we will explore Group By and Order By with retail dataset 

In [ ]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.hadoop.fs._
import org.apache.spark.sql
import org.apache.spark.sql.types._
import org.apache.spark.sql.Encoders
import org.apache.spark.sql.functions.{expr,col,pow}
import org.apache.spark.sql.types.StructField

In [2]:
val conf = new SparkConf().setAppName("Flight status")
val sc = new SparkContext(conf)
val spark = new SQLContext(sc) 

conf = org.apache.spark.SparkConf@40bc738b
sc = org.apache.spark.SparkContext@7f81dc3e
spark = org.apache.spark.sql.SQLContext@25627c05


org.apache.spark.sql.SQLContext@25627c05

# Read  Retail Transactions CSV file with infer schema

In [3]:
val reatil_t = spark.read.format("csv").
            option("header","true").
option("delimiter", ",").
option("inferSchema", "true").
csv("/user/viswatejaster9073/Retail_promotionResponse/Retail_Data_Transactions.csv")

reatil_t = [customer_id: string, trans_date: string ... 1 more field]


[customer_id: string, trans_date: string ... 1 more field]

In [8]:
reatil_t.show(2)

+-----------+----------+-----------+
|customer_id|trans_date|tran_amount|
+-----------+----------+-----------+
|     CS5295| 11-Feb-13|         35|
|     CS4768| 15-Mar-15|         39|
+-----------+----------+-----------+
only showing top 2 rows



# Printing the schema structure of our two data frames 

In [9]:
reatil_t.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- trans_date: string (nullable = true)
 |-- tran_amount: integer (nullable = true)



# Summary of our datasets

### The below summary shows the total count of our columns, mean, standard deviation, Minimum value and Maximum values.
### when you see the mean/standard deviation  for customer_id and trans_date is null which means they are non integer values(you can see schema strcuture at section 2)

In [19]:
reatil_t.describe().show()

+-------+-----------+----------+-----------------+
|summary|customer_id|trans_date|      tran_amount|
+-------+-----------+----------+-----------------+
|  count|     125000|    125000|           125000|
|   mean|       null|      null|        64.991912|
| stddev|       null|      null|22.86000619473492|
|    min|     CS1112| 01-Apr-12|               10|
|    max|     CS9000| 31-Oct-14|              105|
+-------+-----------+----------+-----------------+



In [20]:
reatil_p.describe().show()

+-------+-----------+-------------------+
|summary|customer_id|           response|
+-------+-----------+-------------------+
|  count|       6884|               6884|
|   mean|       null|0.09398605461940732|
| stddev|       null|0.29183051177768093|
|    min|     CS1112|                  0|
|    max|     CS9000|                  1|
+-------+-----------+-------------------+



# find the top 5 frequent customer and less frequent customers using DataFrames
## In this example we are using groupBy and OrderBy to solve our business usecase
### GroupBy - The GROUP BY statement is widely used with aggregate functions (COUNT, MAX, MIN, SUM, AVG) to group the result-set by one or more columns. In our case we need to groupby the customers and counting them as shown below

### OrderBy - We will use this whenever we want to sort our data etither in ascending or decsending order, In our case we need to find the top 5 frequent customers and top 5 least visiting customers

#### In this example we are using count operation in groupBy to check the count of customers 

In [62]:
reatil_t.select("customer_id").groupBy("customer_id").count().orderBy($"count".desc).show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|     CS4424|   39|
|     CS4320|   38|
|     CS3799|   36|
|     CS3805|   35|
|     CS2620|   35|
+-----------+-----+
only showing top 5 rows



In [63]:
reatil_t.select("customer_id").groupBy("customer_id").count().orderBy($"count".asc).show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|     CS8559|    4|
|     CS7716|    4|
|     CS7333|    4|
|     CS8376|    4|
|     CS7224|    4|
+-----------+-----+
only showing top 5 rows



# find the top 5 frequent customer and less frequent customers using <b>Spark SQL <b>
## when we want to use Spark SQL we need to register our DataFrame as a table using createOrReplaceTempView("XXXXXXX") as shown below

In [19]:
reatil_t.createOrReplaceTempView("retail_t") // We are exposing our retail_t dataframe as the table rt

lastException: Throwable = null


In [27]:
spark.sql("select customer_id,count(customer_id) as count from retail_t group by customer_id order by count limit 5").show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|     CS7224|    4|
|     CS8559|    4|
|     CS7716|    4|
|     CS7333|    4|
|     CS8376|    4|
+-----------+-----+



In [29]:
spark.sql("select customer_id,count(customer_id) as count from retail_t group by customer_id order by count desc limit 5").show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|     CS4424|   39|
|     CS4320|   38|
|     CS3799|   36|
|     CS2620|   35|
|     CS3013|   35|
+-----------+-----+



# Find the higest and least transaction dates using DafaFrames
#### In this example we are using sum operation in groupBy to sum all the trascation amount on the perticular dates

In [15]:
reatil_t.groupBy("trans_date").sum().orderBy($"sum(tran_amount)".desc).show(5)

+----------+----------------+
|trans_date|sum(tran_amount)|
+----------+----------------+
| 16-Jul-11|            8791|
| 20-May-14|            8108|
| 15-Nov-12|            8054|
| 11-Aug-11|            7938|
| 18-Sep-11|            7866|
+----------+----------------+
only showing top 5 rows



In [16]:
reatil_t.groupBy("trans_date").sum().orderBy($"sum(tran_amount)".asc).show(5)

+----------+----------------+
|trans_date|sum(tran_amount)|
+----------+----------------+
| 17-Jan-15|            3648|
| 18-Aug-12|            3792|
| 26-Feb-14|            3830|
| 25-Nov-11|            3887|
| 02-Jul-13|            3915|
+----------+----------------+
only showing top 5 rows



# Find the higest and least transaction dates using Spark SQL

In [21]:
spark.sql("select trans_date,sum(tran_amount) as total from retail_t group by trans_date order by total limit 5").show

+----------+-----+
|trans_date|total|
+----------+-----+
| 17-Jan-15| 3648|
| 18-Aug-12| 3792|
| 26-Feb-14| 3830|
| 25-Nov-11| 3887|
| 02-Jul-13| 3915|
+----------+-----+



In [24]:
spark.sql("select trans_date,sum(tran_amount) as total from retail_t group by trans_date order by total desc limit 5").show

+----------+-----+
|trans_date|total|
+----------+-----+
| 16-Jul-11| 8791|
| 20-May-14| 8108|
| 15-Nov-12| 8054|
| 11-Aug-11| 7938|
| 18-Sep-11| 7866|
+----------+-----+



# Find the minimum and maximum transaction dates

In [40]:
reatil_t.groupBy("trans_date").max().show(5)

+----------+----------------+
|trans_date|max(tran_amount)|
+----------+----------------+
| 01-Feb-15|             105|
| 20-Feb-12|             105|
| 22-Jul-12|             105|
| 17-Jan-15|             105|
| 28-Aug-11|             105|
+----------+----------------+
only showing top 5 rows



In [41]:
reatil_t.groupBy("trans_date").min().show(5)

+----------+----------------+
|trans_date|min(tran_amount)|
+----------+----------------+
| 01-Feb-15|              10|
| 20-Feb-12|              10|
| 22-Jul-12|              13|
| 17-Jan-15|              31|
| 28-Aug-11|              10|
+----------+----------------+
only showing top 5 rows



# Find the minimum and maximum transaction dates using Spark SQL

In [30]:
spark.sql("select trans_date,min(tran_amount) as minimum_amt,max(tran_amount) as max_amount from retail_t group by trans_date limit 5").show

+----------+-----------+----------+
|trans_date|minimum_amt|max_amount|
+----------+-----------+----------+
| 01-Feb-15|         10|       105|
| 20-Feb-12|         10|       105|
| 22-Jul-12|         13|       105|
| 17-Jan-15|         31|       105|
| 28-Aug-11|         10|       105|
+----------+-----------+----------+

